In [8]:
import pandas as pd
import numpy as np
import random 

In [204]:
class visitor:
    def __init__(self, vid, numberOfFloors, roomsPerFloor, contagious):
        self.id = vid
        self.contagiousDegree = random.uniform(0.1,0.6) if contagious else 0
        # define number of visit
        numberOfVisits = random.randint(1, numberOfFloors * roomsPerFloor)
        visitedRooms = []
        self.itinerary = [self._decideRoom(numberOfFloors, roomsPerFloor, visitedRooms, None)\
                            for visit in range(1, numberOfVisits + 1)]

    def _decideRoom(self, numberOfFloors, roomsPerFloor, visitedRooms, currentFloor):
            while(True):
                floor = self._decideFloor(numberOfFloors, currentFloor)
                room = random.randint(1, roomsPerFloor)
                if (floor, room) not in visitedRooms:
                    visitedRooms.append((floor, room))
                    return {'floor':floor, 'room':room, 'visitorId':self.id, 'contagiousDegree':self.contagiousDegree}

    def _decideFloor(self, numberOfFloors: int, currentFloor: int):
            if currentFloor==None:  # no current floor, has to be first room in itinerary             
                # decide start floor
                return random.randint(1, numberOfFloors)
            else:
                # decide if stay in current floor or move to another floor
                stayInFloor = random.randint(0, 1)
                if stayInFloor == 1:
                    # stay in current floor
                    return currentFloor

                # else, move to another floor
                # decide if use elevator
                useElevator = random.randint(0, 1)
                if useElevator == 1:
                    # use elevator and move to any other floor in building (excpet current floor)
                    while (True):
                        nextFloor = random.randint(1, numberOfFloors)
                        if nextFloor != currentFloor:
                            return nextFloor

                # else, don't use elevator, use stairs instead
                goOneFloorUpstairs = random.randint(0, 1)
                if goOneFloorUpstairs == 0 and currentFloor != 1:
                    # don't go upstairs, go downstairs one floor
                    return currentFloor - 1
                    # else, go upstairs one floor
                return currentFloor + 1

In [301]:
roomsPerFloor = 4
numberOfFloors = 3
numberOfVisitor= 5

building = [{'floor':floor, 'room':room, 'susceptability':random.uniform(0,1)}\
            for floor in range(1,numberOfFloors+1)\
            for room in range(1,roomsPerFloor+1)]


contagiousVisitor = random.randint(1,numberOfVisitor)

itineraries = [visitor(vid,numberOfFloors,roomsPerFloor,contagiousVisitor==vid).itinerary\
          for vid in range(1,numberOfVisitor+1)]

infected = [{'floor':visit['floor'], 'room':visit['room']}\
            for itinerary in itineraries
            for visit in itinerary
            for r in building
            if visit['contagiousDegree']>0 and
               r['floor']==visit['floor'] and
               r['room']==visit['room'] and
               r['susceptability']+visit['contagiousDegree'] > 0.9]


In [302]:
print('building')
pd.DataFrame(building)

building


,floor,room,susceptability
0,1,1,0.190944
1,1,2,0.630669
2,1,3,0.595357
3,1,4,0.122601
4,2,1,0.532837
5,2,2,0.280506
6,2,3,0.880787
7,2,4,0.997258
8,3,1,0.487334
9,3,2,0.263014


In [303]:
print('itineraries')
print("\n".join(["floor: {} room:{} vid: {} contagiousDegree: {}".format(
                 v["floor"], v["room"], v["visitorId"], v["contagiousDegree"]) 
                 for itinerary in itineraries
                 for v in itinerary
                 #if v["contagiousDegree"] >0
                ]))

itineraries
floor: 3 room:3 vid: 1 contagiousDegree: 0
floor: 1 room:4 vid: 1 contagiousDegree: 0
floor: 2 room:3 vid: 1 contagiousDegree: 0
floor: 3 room:1 vid: 1 contagiousDegree: 0
floor: 3 room:2 vid: 1 contagiousDegree: 0
floor: 1 room:1 vid: 1 contagiousDegree: 0
floor: 2 room:3 vid: 2 contagiousDegree: 0
floor: 2 room:2 vid: 2 contagiousDegree: 0
floor: 3 room:4 vid: 2 contagiousDegree: 0
floor: 2 room:1 vid: 2 contagiousDegree: 0
floor: 1 room:2 vid: 2 contagiousDegree: 0
floor: 3 room:1 vid: 2 contagiousDegree: 0
floor: 2 room:4 vid: 2 contagiousDegree: 0
floor: 3 room:2 vid: 2 contagiousDegree: 0
floor: 1 room:3 vid: 2 contagiousDegree: 0
floor: 2 room:1 vid: 3 contagiousDegree: 0.36013496065457284
floor: 1 room:1 vid: 3 contagiousDegree: 0.36013496065457284
floor: 1 room:2 vid: 3 contagiousDegree: 0.36013496065457284
floor: 2 room:2 vid: 3 contagiousDegree: 0.36013496065457284
floor: 2 room:4 vid: 3 contagiousDegree: 0.36013496065457284
floor: 1 room:4 vid: 3 contagiousDegre

In [304]:
print('infected')
pd.DataFrame(infected)

infected


,floor,room
0,1,2
1,2,4


In [305]:
suspiciousVisitors = set([v['visitorId']
                      for visits in itineraries 
                      for v in visits 
                      for i in infected 
                      if v['floor']==i['floor']
                         and v['room']==i['room']])
suspiciousVisitors 

{2, 3, 4, 5}

In [306]:
from collections import Counter
 
countInfectedVisits = Counter([v['visitorId']
                      for visits in itineraries 
                      for v in visits 
                      for i in infected 
                      if v['floor']==i['floor']
                         and v['room']==i['room']])
countInfectedVisits 

Counter({2: 2, 3: 2, 4: 1, 5: 2})

In [307]:
countVisits = Counter([v['visitorId']
                      for visits in itineraries 
                      for v in visits])
countVisits.items()

dict_items([(1, 6), (2, 9), (3, 6), (4, 1), (5, 6)])

In [308]:
ratios = [(vid,vi/v)
         for vid, v in countVisits.items()
         for visitorId, vi in countInfectedVisits.items()
         if vid==visitorId]
ratios

[(2, 0.2222222222222222),
 (3, 0.3333333333333333),
 (4, 1.0),
 (5, 0.3333333333333333)]

In [309]:
df = pd.DataFrame(ratios)
mainSuspect = df[df.iloc[:,1]==max(df.iloc[:,1])]
mainSuspect

,0,1
2,4,1.0
